# Import Keras InceptionV3 Model with Pre-Trained Weights File

This notebook shows several methods that you can use with SAS DLPy to import a Keras model and create a SAS Deep Learning  model, both with and without pre-trained weights. This example imports and weights an InceptionV3 CNN image classification model, but the techniques in this example will work to import and configure other Keras library models as well.

Different scenarios in the notebook show how to import the Keras InceptionV3 model from a repository, with and without pre-trained weights stored in HDF5 (.h5) format. 

In the first scenario, the example imports the unweighted Keras InceptionV3 model and downloads the weights file from a repository in separate steps. Then SAS DLPy is used to convert the imported Keras InceptionV3 model to a SAS Deep Learning InceptionV3 model. The final step uses DLPy and the downloaded HDF5 weights file to update the unweighted SAS Deep Learning InceptionV3 model to a SAS Deep Learning InceptionV3 model that includes pre-trained weights. 

[Initialize Required Python and SAS DLPy Utilities](#Initialize)

[Import Keras InceptionV3 Model from Repository](#Import_Keras_InceptionV3_Model_from_Repository)

[Start a SAS CAS Session](#Start_a_SAS_CAS_Session)

[Download the InceptionV3 Model Weights File](#Download_the_InceptionV3_Model_Weights_File)

[Convert Keras InceptionV3 Model to SAS Deep Learning Model](#Convert_Keras_InceptionV3_Model_to_SAS_Deep_Learning_Model)

[Load Pre-Trained Model Weights from Keras HDF5 File](#Load_Pre-Trained_Model_Weights_from_Keras_HDF5_File)

[Alternative Approaches](#Alternative_Approaches) 

<a id='Initialize'></a>
### Initialize Required Python and SAS DLPy Utilities

This step initializes Python, SAS DLPy, and Keras modules that are referenced later in the notebook. 

First, import the os module that is used with path references in the code. Next, import SAS SWAT. SWAT is the SAS Scripting Wrapper for Analytics Transfer. SWAT is the Python interface to SAS CAS that enables you to load data into memory and apply CAS actions to the data on the server.

<b>Note:</b> For more information about starting a CAS session with the SWAT package, see https://sassoftware.github.io/python-swat/getting-started.html.

The SAS DLPy Model and ImageTable modules are needed for InceptionV3 models.

The Keras library is loaded for Keras usage. The Keras inception_v3 module architecture will be referenced.


In [1]:
import os
import swat
from dlpy import Model
from dlpy.images import ImageTable

import keras
from keras.applications import inception_v3

Using TensorFlow backend.


<a id='Import_Keras_InceptionV3_Model_from_Repository'></a>
<a id='section1'></a>
### Import Keras InceptionV3 Model from Repository

Next, use the Keras library to create an unweighted Keras InceptionV3 model that uses an input shape of (299, 299, 3). and load the model with an imagenet weight set that will be downloaded by the code. The trained model weights in HDF5 format are downloaded in a later step.

In [2]:
# Use the Keras library to import an unweighted InceptionV3 model  
keras_inception_model = inception_v3.InceptionV3(weights=None, input_shape=(299, 299, 3))

Use the Keras model 'summary()' function to display the structure of the unweighted, newly created `keras_inception_model`:

In [3]:
# Use the Keras model summary() function to display the structure of the new InceptionV3 model.
# This model has no weights.

print(keras_inception_model.summary())

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

Now it is time to start SAS CAS and gain access to the DLPy API tools:

<a id='Start_a_SAS_CAS_Session'></a>
### Start a SAS CAS Session

In [4]:
# Log into SAS CAS with your host and port information

s = swat.CAS('your-host.unx.xyz.com', 12345)


# Required for DLPY prior to v1.2; otherwise can be deleted
s.loadactionset('deepLearn')


NOTE: Added action set 'deepLearn'.


[actionset]

 'deepLearn'

+ Elapsed: 0.00255s, user: 0.00035s, sys: 0.00219s, mem: 0.205mb

<a id='Download_the_InceptionV3_Model_Weights_File'></a>
### Download the Keras InceptionV3 Model Weights File


The next step downloads the model weights file for the Keras InceptionV3 model. The Keras InceptionV3 model weights in .h5 format are from [Francois Chollet's Deep-Learning-Models](https://github.com/fchollet/deep-learning-models) repository on GitHub. Copyright and permission notices to use the repository's InceptionV3 model data are granted and detailed in the [MIT License](https://github.com/fchollet/deep-learning-models/blob/master/LICENSE).

To directly download the InceptionV3 model weights file `inception_v3_weights_tf_dim_ordering_tf_kernels.h5` used in this example, go [here](https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels.h5). The file is approximately 94MB in size.

Save the downloaded model weights to a location that your server can access. The example code in this notebook uses this fictional server path for the stored model weight files:

`'\\path-to\your-server\download-directory\inception_v3_weights_tf_dim_ordering_tf_kernels.h5'`

Your server path for the saved InceptionV3 model weights will be different.



<a id='Convert_Keras_InceptionV3_Model_to_SAS_Deep_Learning_Model'></a>
### Convert Keras InceptionV3 Model to SAS Deep Learning Model 

The next step uses SAS DLPy to convert the Keras InceptionV3 model `keras_inception_model` to a SAS Deep Learning InceptionV3 model named `dlpy_inception_model`. The output table is named `DLPy_inceptionv3`. 

In [5]:
# Specify the server directory where you stored the downloaded pre-trained .h5 model weight file. 

server_weight_root = '\\path-to\your-server\download-directory'

dlpy_inception_model,use_gpu = Model.from_keras_model(conn=s, 
                                                      keras_model=keras_inception_model, 
                                                      output_model_table='DLPy_inceptionv3',
                                                      # we do not load the InceptionV3 model  
                                                      # weights in this step, but we could...
                                                      include_weights=False, 
                                                      )

INFO: Setting channel mean values to ImageNet means
NOTE: Model table is attached successfully!
NOTE: Model is named to "dlpy_inceptionv3" according to the model name in the table.


We can use the DLPy `print_summary()` command to verify that a SAS Deep Learning model named `dlpy_inception_model` was created:

In [6]:
dlpy_inception_model.print_summary()



,Layer Id,Layer,Type,Kernel Size,Stride,Activation,Output Size,Number of Parameters
0,0,input_1,input,,,None,"(299, 299, 3)","(0, 0)"
1,1,conv2d_1,convo,"(3, 3)","(2.0, 2.0)",Identity,"(150, 150, 32)","(864, 0)"
2,2,batch_normalization_1,batchnorm,,,Rectifier,"(150, 150, 32)","(0, 64)"
3,3,conv2d_2,convo,"(3, 3)","(1.0, 1.0)",Identity,"(150, 150, 32)","(9216, 0)"
4,4,batch_normalization_2,batchnorm,,,Rectifier,"(150, 150, 32)","(0, 64)"
...,...,...,...,...,...,...,...,...
215,215,concatenate_2,concat,,,Auto,"(10, 10, 768)","(0, 0)"
216,216,mixed10,concat,,,Auto,"(10, 10, 2048)","(0, 0)"
217,217,avg_pool,pool,"(8, 8)","(1.0, 1.0)",Mean,"(10, 10, 2048)","(0, 0)"
218,218,predictions,output,"(204800, 1000)",,Softmax,1000,"(204800000, 1000)"


<a id='Load_Pre-Trained_Model_Weights_from_Keras_HDF5_File'></a>
### Load Pre-Trained Model Weights from Keras HDF5 File
In this step, you integrate the pre-trained Keras model weights (that you downloaded separately in .h5 format) into the new model `dlpy_inception_model` using the SAS DLPy `load_weights_from_keras()` function. 

In [7]:
                                                
# Load the pre-trained weights that you saved to the server using DLPy                                             
dlpy_inception_model.load_weights_from_keras(path='\\path-to\your-server\download-directory\inception_v3_weights_tf_dim_ordering_tf_kernels.h5')
                                             
                                             # 
 


NOTE: no dataspec(s) provided - creating image classification model.
NOTE: Model weights attached successfully!


You now have a pre-trained, weighted SAS Deep Learning InceptionV3 image classification network that was imported from an open-source Keras model library repository and converted using SAS DLPy. 

That was one way to solve that model importation and weighting task. Are there other ways? Yes, there are. See the next section for more details.

<a id='Alternative_Approaches'></a>
### Alternative Approaches 

So far in this example notebook, a Keras InceptionV3 model was imported and pre-trained model weights were downloaded from an open-source Keras repository on GitHub. SAS DLPy was used to convert the unweighted Keras InceptionV3 model into a SAS Deep Learning InceptionV3 model, then DLPy used the InceptionV3 HDF5 (.h5) weights file to integrate the pre-trained weights into the model.

Here are some other ways that you can use open-source Keras library models with SAS DLPy to create SAS Deep Learning models with pre-trained weights. You might find the streamlined approaches below to be preferable because they are more compact:

In [Import Keras InceptionV3 Model with Pre-Trained Weights and Convert to SAS Deep Learning Model with Pre-Trained Weights](#alt1), you import the Keras model *built with pre-trained weights* from the repository. Then you use DLPy to convert the imported Keras model with pre-trained weights into a SAS Deep Learning model with pre-trained weights. 

In [Import Keras InceptionV3 Model, Download Model Weights and Convert to SAS Deep Learning Model with Pre-Trained Weights](#alt2), you import the Keras model without weights, and you download the pre-trained HDF5 model weights separately. Then you use a single block of in SAS DLPy to convert the Keras model and weights file into a SAS Deep Learning model with pre-trained weights.   


<a id='alt1'></a>
#### Import Keras InceptionV3 Model with Pre-Trained Weights and Convert to SAS Deep Learning Model with Pre-Trained Weights

This notebook section shows how you can download the InceptionV3 model from Chollet's GitHub respository complete with model weights. This method does not require you to download the pretrained model weights separately. The `weights='imagenet'` setting in the command imports the built model complete with pre-trained weights. Then the weighted Keras InceptionV3 model can be converted to a weighted SAS Deep Learning format in a simple step using SAS DLPy.

In [8]:
# This step imports a new Keras InceptionV3 model with pre-trained weights from a GitHub repository in one pass.
keras_inception_model2 = inception_v3.InceptionV3(weights='imagenet')

Now the Keras model `keras_inception_model2` with pre-trained weights can be converted to SAS Deep Learning format and named `dlpy_inception_model2` using SAS DLPy. The model output table in CAS will be called `DLPy_inceptionv3_2`:

In [9]:
dlpy_inception_model2,use_gpu = Model.from_keras_model(conn=s, 
                                                      keras_model=keras_inception_model2, 
                                                      output_model_table='DLPy_inceptionv3_2',
                                                      )

INFO: Setting channel mean values to ImageNet means
NOTE: Model table is attached successfully!
NOTE: Model is named to "dlpy_inceptionv3_2" according to the model name in the table.


We can take a look at the structure of the new model `dlpy_inception_model2` using DLPy's `print_summary()` function:

In [10]:
dlpy_inception_model2.print_summary()

,Layer Id,Layer,Type,Kernel Size,Stride,Activation,Output Size,Number of Parameters
0,0,input_2,input,,,None,"(299, 299, 3)","(0, 0)"
1,1,conv2d_95,convo,"(3, 3)","(2.0, 2.0)",Identity,"(150, 150, 32)","(864, 0)"
2,2,batch_normalization_95,batchnorm,,,Rectifier,"(150, 150, 32)","(0, 64)"
3,3,conv2d_96,convo,"(3, 3)","(1.0, 1.0)",Identity,"(150, 150, 32)","(9216, 0)"
4,4,batch_normalization_96,batchnorm,,,Rectifier,"(150, 150, 32)","(0, 64)"
...,...,...,...,...,...,...,...,...
215,215,concatenate_4,concat,,,Auto,"(10, 10, 768)","(0, 0)"
216,216,mixed10,concat,,,Auto,"(10, 10, 2048)","(0, 0)"
217,217,avg_pool,pool,"(8, 8)","(1.0, 1.0)",Mean,"(10, 10, 2048)","(0, 0)"
218,218,predictions,output,"(204800, 1000)",,Softmax,1000,"(204800000, 1000)"


<a id='alt2'></a>
#### Import Keras InceptionV3 Model, Download Model Weights and Convert to Weighted SAS Deep Learning Model

This notebook section imports an unweighted InceptionV3 model from a Keras library and names the model `keras_inception_model3`. The model is configured to accept input images in the shape (299, 299, 3). The separately downloaded weights file is integrated with the SAS Deep Learning model during the DLPy conversion.

In [11]:
# Use the Keras library to import an unweighted InceptionV3 model  
keras_inception_model3 = inception_v3.InceptionV3(weights=None, input_shape=(299, 299, 3))

 

If you have not already done so in the first part of this notebook, you can download the InceptionV3 model weights file inception_v3_weig hts_tf_dim_ordering_tf_kernels.h5 (94MB) [here](https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels.h5). 

Save the downloaded .h5 model weights file to a location that your server can access. The example code uses this fictitions server path for the stored model weights:

'\path-to\your-server\download-directory\inception_v3_weights_tf_dim_ordering_tf_kernels.h5'

Your server path will be different.

The code below uses SAS DLPy to convert the keras InceptionV3 classification model `keras_inception_model3` into a SAS Deep Learning InceptionV3 model with pre-trained weights named `dlpy_inception_model3`.

In [12]:
# Specify the server directory where you stored the downloaded pre-trained .h5 model weight file. 

server_weight_root = '\\path-to\your-server\Download-Directory'

dlpy_inception_model3,use_gpu = Model.from_keras_model(conn=s, 
                                                      keras_model=keras_inception_model3, 
                                                      output_model_table='DLPy_inceptionv3_3',
                                                      include_weights=False, 
                                                      input_weights_file=os.path.join(server_weight_root,
                                                                                      "inception_v3_weights_tf_dim_ordering_tf_kernels.h5"
                                                                                     ) 
                                                      )

INFO: Setting channel mean values to ImageNet means
NOTE: Model table is attached successfully!
NOTE: Model is named to "dlpy_inceptionv3_3" according to the model name in the table.


You can use the DLPy print_summary() command to validate the newly created SAS Deep Learning model `dlpy_inception_model3`:

In [13]:
dlpy_inception_model3.print_summary()

,Layer Id,Layer,Type,Kernel Size,Stride,Activation,Output Size,Number of Parameters
0,0,input_3,input,,,None,"(299, 299, 3)","(0, 0)"
1,1,conv2d_189,convo,"(3, 3)","(2.0, 2.0)",Identity,"(150, 150, 32)","(864, 0)"
2,2,batch_normalization_189,batchnorm,,,Rectifier,"(150, 150, 32)","(0, 64)"
3,3,conv2d_190,convo,"(3, 3)","(1.0, 1.0)",Identity,"(150, 150, 32)","(9216, 0)"
4,4,batch_normalization_190,batchnorm,,,Rectifier,"(150, 150, 32)","(0, 64)"
...,...,...,...,...,...,...,...,...
215,215,concatenate_6,concat,,,Auto,"(10, 10, 768)","(0, 0)"
216,216,mixed10,concat,,,Auto,"(10, 10, 2048)","(0, 0)"
217,217,avg_pool,pool,"(8, 8)","(1.0, 1.0)",Mean,"(10, 10, 2048)","(0, 0)"
218,218,predictions,output,"(204800, 1000)",,Softmax,1000,"(204800000, 1000)"


Despite the difference in methods used to create and weight the final SAS Deep Learning InceptionV3 image classification model, all of the outcomes show identical deep learning model structures.